In [1]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path="../models/best.pt")

Using cache found in /home/misge/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-2-15 Python-3.12.1 torch-2.2.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7072156 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [2]:
import os, sys, glob
import pandas as pd

rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from scripts.symbol_detector import SymbolDetection

In [3]:
sd = SymbolDetection(model, 0.25)

In [4]:
sd.query('../datasets/new_previews/002dbbd85ef3fe6a2e7d0754fb9f9a1a.png')

[{'Box': {'x': 0.3053763440860215,
   'y': 0.5764075067024129,
   'width': 465,
   'height': 373},
  'conf': 0.8018277287483215,
  'label': 'Text Elements'}]

In [5]:
import cv2

image_path = '../datasets/new_previews/002dbbd85ef3fe6a2e7d0754fb9f9a1a.png'
image = cv2.imread(image_path)

if image is None:
    print(f"Error: Unable to load image at {image_path}")
    exit()

height, width, channels = image.shape
print(f"Image Dimensions: Width={width}, Height={height}, Channels={channels}")

cv2.rectangle(image, (0, 0), (width-1, height-1), (255, 0, 0), 2)

cv2.imshow('Image with Dimensions', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Image Dimensions: Width=600, Height=500, Channels=3


In [5]:
folder_path = '../datasets/new_previews/'

output = []
image_files = glob.glob(os.path.join(folder_path, '*.jpg')) + glob.glob(os.path.join(folder_path, '*.png'))

for image_file in image_files:
    coords = sd.query(image_file)
    coords.append(
        {
            'img-name': image_file.split('/')[-1]
        }
    )
    output.append(coords)

In [6]:
output[:5]

[[{'Box': {'x': 0.7843803056027164,
    'y': 0.6068965517241379,
    'width': 589,
    'height': 145},
   'conf': 0.3523620069026947,
   'label': 'Text Elements'},
  {'Box': {'x': 0.7781512605042017,
    'y': 0.6137931034482759,
    'width': 595,
    'height': 145},
   'conf': 0.3773348927497864,
   'label': 'Logo'},
  {'Box': {'x': 0.5994152046783626,
    'y': 0.018867924528301886,
    'width': 342,
    'height': 106},
   'conf': 0.4615219831466675,
   'label': 'Interactive Elements'},
  {'Box': {'x': 0.6185567010309279,
    'y': 0.8616187989556136,
    'width': 485,
    'height': 383},
   'conf': 0.49012601375579834,
   'label': 'Text Elements'},
  {'Box': {'x': 0.5747126436781609,
    'y': 0.03773584905660377,
    'width': 348,
    'height': 106},
   'conf': 0.5754431486129761,
   'label': 'Logo'},
  {'img-name': '427bf316b866d8b20a30866426c66e09.png'}],
 [{'Box': {'x': 0.4254807692307692, 'y': 0.88, 'width': 416, 'height': 475},
   'conf': 0.7730779051780701,
   'label': 'Call-To-A

In [7]:
df = pd.read_csv('../datasets/performance_data.csv')
df

,game_id,preview_link,ER,CTR
0,bfbf7a2b7ac635e67877b1ab87fd6629,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.209269,0.058438
1,945010afbf9a55bbdffcb0895f946155,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.274552,0.074731
2,e68e20f592457b875ce29757ab855dfe,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.103688,0.042228
3,adunit-nestle-purina-friskies-mob,https://s3.ap-southeast-1.amazonaws.com/a.futu...,0.136963,0.005393
4,adunit-lionsgate-uwomt-user-slider-sensory-vid...,https://s3.us-west-1.amazonaws.com/a.futureadl...,0.114208,0.016443
...,...,...,...,...
900,2443bd3b890824acecbe32659b9a1246,https://s3.eu-west-1.amazonaws.com/a.futureadl...,0.365997,0.008790
901,7ed73c1cc404f9956d26c5f58de72b2f,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.103055,0.043215
902,90455f5ace15860874965eabd081b6f2,https://s3.ap-southeast-1.amazonaws.com/a.futu...,0.191425,0.038692
903,adunit-chevy-hotspot-cpe-av-mpu,https://s3.us-east-1.amazonaws.com/a.futureadl...,0.073058,0.000057


In [18]:
rows = []
for image in output:
    game_id = None 
    single_image = []
    for row_data in image:
        if "img-name" in row_data:
            game_id = row_data['img-name'][:-4]
    
    temp = {'game_id': game_id}
    for row_data in image:
        if "img-name" in row_data:
            continue 
        
        if 'Box' in row_data:
            for attribute in ['x', 'y', 'width', 'height']:
                temp[f"{row_data['label']}_{attribute}"] = row_data['Box'][attribute]
            temp[f'{row_data['label']}_conf'] = row_data['conf']
    
    rows.append(temp)


In [19]:
rows[:5]

[{'game_id': '427bf316b866d8b20a30866426c66e09',
  'Text Elements_x': 0.6185567010309279,
  'Text Elements_y': 0.8616187989556136,
  'Text Elements_width': 485,
  'Text Elements_height': 383,
  'Text Elements_conf': 0.49012601375579834,
  'Logo_x': 0.5747126436781609,
  'Logo_y': 0.03773584905660377,
  'Logo_width': 348,
  'Logo_height': 106,
  'Logo_conf': 0.5754431486129761,
  'Interactive Elements_x': 0.5994152046783626,
  'Interactive Elements_y': 0.018867924528301886,
  'Interactive Elements_width': 342,
  'Interactive Elements_height': 106,
  'Interactive Elements_conf': 0.4615219831466675},
 {'game_id': 'b12b03c2dec5e71fe630d3b06062982a',
  'Call-To-Action (CTA) Button_x': 0.4254807692307692,
  'Call-To-Action (CTA) Button_y': 0.88,
  'Call-To-Action (CTA) Button_width': 416,
  'Call-To-Action (CTA) Button_height': 475,
  'Call-To-Action (CTA) Button_conf': 0.7730779051780701,
  'Logo_x': 0.5771276595744681,
  'Logo_y': 0.09090909090909091,
  'Logo_width': 376,
  'Logo_height': 

In [20]:
df = pd.DataFrame(rows)

In [21]:
df.head()

,game_id,Text Elements_x,Text Elements_y,Text Elements_width,Text Elements_height,...,Product Image_x,Product Image_y,Product Image_width,Product Image_height,Product Image_conf
0,427bf316b866d8b20a30866426c66e09,0.618557,0.861619,485.0,383.0,...,NaN,NaN,NaN,NaN,NaN
1,b12b03c2dec5e71fe630d3b06062982a,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2,cafcf13bef3457ebed7e08e02fa49252,0.234818,0.223776,494.0,143.0,...,NaN,NaN,NaN,NaN,NaN
3,94bc1e8d137186644fa3baa53185a331,0.089789,0.894057,568.0,774.0,...,NaN,NaN,NaN,NaN,NaN
4,07a8072d594af3c04ef47e8722fc1b38,0.125285,0.731429,439.0,175.0,...,NaN,NaN,NaN,NaN,NaN


In [22]:
df.value_counts()

game_id                           Text Elements_x  Text Elements_y  Text Elements_width  Text Elements_height  Text Elements_conf  Logo_x    Logo_y    Logo_width  Logo_height  Logo_conf  Interactive Elements_x  Interactive Elements_y  Interactive Elements_width  Interactive Elements_height  Interactive Elements_conf  Call-To-Action (CTA) Button_x  Call-To-Action (CTA) Button_y  Call-To-Action (CTA) Button_width  Call-To-Action (CTA) Button_height  Call-To-Action (CTA) Button_conf  Product Image_x  Product Image_y  Product Image_width  Product Image_height  Product Image_conf
29f4c491737ce1cd367ff019e7b2ccb7  0.073944         0.739816         568.0                761.0                 0.459461            0.534606  0.335938  419.0       128.0        0.622546   0.695312                0.223022                256.0                       139.0                        0.342961                   0.422434                       0.920838                       419.0                              85

In [23]:
df.to_csv('../datasets/labeled.csv', index=False)